In [43]:
# Read in libraries
import pandas as pd
import numpy as np
from collections import defaultdict
import itertools
from typing import Dict, List, Tuple

# Read the CSV file that contains season lineup data
vt24 = pd.read_csv("C:/Users/bwdea/OneDrive/Documents/Basketball/Virginia Tech/Lineups/Games/VT-All Games.csv")

# Define roster with positions, minutes, and games played
roster_data = {
    'Tobi Lawal': {'positions': [4, 5], 'minutes': 791, 'games': 30},
    'Jaden Schutt': {'positions': [2, 3], 'minutes': 840, 'games': 32},
    'Tyler Johnson': {'positions': [3, 4], 'minutes': 816, 'games': 32},
    'Jaydon Young': {'positions': [2, 3], 'minutes': 714, 'games': 32},
    'Brandon Rechsteiner': {'positions': [1, 2], 'minutes': 709, 'games': 32},
    'Ben Burnham': {'positions': [4], 'minutes': 704, 'games': 32},
    'Mylyjael Poteat': {'positions': [5], 'minutes': 674, 'games': 31},
    'Ben Hammond': {'positions': [1, 2], 'minutes': 566, 'games': 29},
    'Rodney Brown Jr.': {'positions': [3, 2], 'minutes': 274, 'games': 18},
    'Patrick Wessler': {'positions': [5], 'minutes': 331, 'games': 31},
    'Connor Serven': {'positions': [4, 3, 5], 'minutes': 41, 'games': 16},
    'Ryan Jones Jr.': {'positions': [4, 5], 'minutes': 10, 'games': 5},
    'Connor Venable': {'positions': [2, 3], 'minutes': 3, 'games': 3},
    'Peter Carr': {'positions': [3], 'minutes': 3, 'games': 3}
}

# Extract separate dictionaries for easy access
player_positions = {name: data['positions'] for name, data in roster_data.items()}
minutes_all = {name: data['minutes'] for name, data in roster_data.items()}
games_played = {name: data['games'] for name, data in roster_data.items()}

# Get all player columns from the lineup data
player_columns = ['Player1', 'Player2', 'Player3', 'Player4', 'Player5']

# Calculate plus/minus for each player
player_stats = {}
for _, row in vt24.iterrows():
    plus_minus = row['+/-']
    for col in player_columns:
        player_name = row[col]
        if player_name not in player_stats:
            player_stats[player_name] = 0
        player_stats[player_name] += plus_minus

# Filter to only players with at least 40 TOTAL minutes
MIN_MINUTES = 40

roster = [name for name, mins in minutes_all.items() if mins >= MIN_MINUTES]
minutes_dict = {name: minutes_all[name] for name in roster}

# Calculate plus/minus per minute for filtered players
plusminuses = {player: player_stats.get(player, 0) / minutes_dict[player] 
               for player in roster}

# Calculate bounds based on actual minutes per game for each player
bounds = {}
for player in roster:
    avg_mpg = minutes_dict[player] / games_played[player]
    lower = max(0, avg_mpg - 5)
    upper = avg_mpg + 5
    bounds[player] = (lower, upper)

# Calculate target minutes for each player (midpoint of their bounds)
target_minutes = {}
for player in roster:
    target_minutes[player] = (bounds[player][0] + bounds[player][1]) / 2

# Sort players by plus/minus per minute (best to worst)
ranked_players = sorted(roster, key=lambda p: plusminuses[p], reverse=True)

print("Player Rankings by +/- per minute:")
print("=" * 70)
for i, player in enumerate(ranked_players, 1):
    avg_mpg = minutes_dict[player] / games_played[player]
    print(f"{i:2}. {player:20} {plusminuses[player]:+.4f} per min  "
          f"({avg_mpg:.1f} mpg over {games_played[player]} games)")
    print(f"     Minute range: {bounds[player][0]:.1f}-{bounds[player][1]:.1f}")
    positions_str = ", ".join(str(p) for p in player_positions.get(player, []))
    print(f"     Can play positions: {positions_str}")
print()

# Assign players to positions by iterating through players in rank order
# Each player gets assigned to their positions in priority order until exhausted

# Track remaining minutes needed at each position
position_minutes_remaining = {pos: 40.0 for pos in range(1, 6)}

# Track remaining minutes available for each player
player_minutes_remaining = {player: bounds[player][1] for player in roster}

# Track assignments: position -> [(player, minutes)]
position_assignments = {pos: [] for pos in range(1, 6)}

# Iterate through players from best to worst
for player in ranked_players:
    if player_minutes_remaining[player] < 0.01:
        continue
    
    # Try to assign this player to their positions in priority order
    for position in player_positions[player]:
        if player_minutes_remaining[player] < 0.01:
            break  # Player exhausted
        
        if position_minutes_remaining[position] < 0.01:
            continue  # Position already full
        
        # Determine how many minutes to assign
        minutes_to_assign = min(
            player_minutes_remaining[player],
            position_minutes_remaining[position]
        )
        
        # Make the assignment
        position_assignments[position].append((player, minutes_to_assign))
        player_minutes_remaining[player] -= minutes_to_assign
        position_minutes_remaining[position] -= minutes_to_assign

# Follow the position assignments as substitution queues
# Each position has a queue of (player, minutes) pairs
# When a player's minutes run out at a position, substitute in the next player from that position's queue

# Create position queues
position_queues = {}
for position in range(1, 6):
    position_queues[position] = position_assignments[position].copy()

# Track current lineup and when each player needs to be subbed out
current_lineup = {}  # position -> (player, minutes_remaining_at_position)
for position in range(1, 6):
    if len(position_queues[position]) > 0:
        player, mins = position_queues[position].pop(0)
        current_lineup[position] = (player, mins)

# Track lineups over time
lineups = []
game_time = 0.0

while game_time < 40.0:
    # Find the next substitution time (when someone needs to be replaced)
    next_sub_time = None
    next_sub_position = None
    
    for position, (player, minutes_left) in current_lineup.items():
        if next_sub_time is None or minutes_left < next_sub_time:
            next_sub_time = minutes_left
            next_sub_position = position
    
    if next_sub_time is None or next_sub_time <= 0:
        break
    
    # Don't go past 40 minutes
    duration = min(next_sub_time, 40.0 - game_time)
    
    # Build lineup dict
    lineup_dict = {pos: player for pos, (player, _) in current_lineup.items()}
    
    # Check for duplicates and fix if found
    player_count = {}
    for pos, player in lineup_dict.items():
        if player not in player_count:
            player_count[player] = []
        player_count[player].append(pos)
    
    # If any player is at multiple positions, replace them at non-primary positions
    for player, positions in player_count.items():
        if len(positions) > 1:
            primary_pos = player_positions[player][0]
            for pos in positions:
                if pos != primary_pos:
                    # Try to replace from position queue first
                    replacement_found = False
                    
                    # Try position queue
                    while len(position_queues[pos]) > 0 and not replacement_found:
                        replacement_player, replacement_mins = position_queues[pos].pop(0)
                        if replacement_player not in lineup_dict.values():
                            current_lineup[pos] = (replacement_player, replacement_mins)
                            lineup_dict[pos] = replacement_player
                            replacement_found = True
                    
                    # If queue is empty, find ANY player who can play this position
                    if not replacement_found:
                        for candidate in roster:
                            if pos in player_positions[candidate] and candidate not in lineup_dict.values():
                                # Use remaining game time as available minutes
                                remaining_time = 40.0 - game_time
                                current_lineup[pos] = (candidate, remaining_time)
                                lineup_dict[pos] = candidate
                                replacement_found = True
                                break
    
    # Record the fixed lineup
    lineups.append({
        'duration': duration,
        'lineup': lineup_dict,
        'rating': sum(plusminuses[player] for player in lineup_dict.values())
    })
    
    # Update game time
    game_time += duration
    
    # Deduct time from all players in current lineup
    for position in current_lineup:
        player, minutes_left = current_lineup[position]
        current_lineup[position] = (player, minutes_left - duration)
    
    # If 40 minutes reached stop
    if game_time >= 39.99:
        break
    
    # Substitute out the exhausted player(s)
    for position, (player, minutes_left) in list(current_lineup.items()):
        if minutes_left < 0.01:
            # This player is exhausted at this position
            if len(position_queues[position]) > 0:
                # Sub in next player from this position's queue
                next_player, next_mins = position_queues[position].pop(0)
                current_lineup[position] = (next_player, next_mins)
            else:
                break

# Print lineups
print("\n" + "=" * 70)
print("LINEUP ROTATIONS:")
print("=" * 70)

for i, lineup_info in enumerate(lineups, 1):
    print(f"\nLineup {i}: {lineup_info['duration']:.1f} minutes (Rating: {lineup_info['rating']:+.4f} per min)")
    print("-" * 70)
    for pos in range(1, 6):
        player = lineup_info['lineup'][pos]
        print(f"  Position {pos}: {player:20} (+/- per min: {plusminuses[player]:+.4f})")
    expected_plus_minus = lineup_info['duration'] * lineup_info['rating']
    print(f"  Expected +/- for this stint: {expected_plus_minus:+.2f}")

# Calculate total minutes per player across all lineups
player_total_minutes = defaultdict(float)
player_position_breakdown = defaultdict(lambda: defaultdict(float))

for lineup_info in lineups:
    for pos, player in lineup_info['lineup'].items():
        player_total_minutes[player] += lineup_info['duration']
        player_position_breakdown[player][pos] += lineup_info['duration']

print("\nMinutes by player:")
for player in sorted(player_total_minutes.keys(), 
                    key=lambda p: player_total_minutes[p], reverse=True):
    lower = bounds[player][0]
    upper = bounds[player][1]
    scheduled = player_total_minutes[player]
    avg_mpg = minutes_dict[player] / games_played[player]
    print(f"  {player:20}: {scheduled:5.1f} min (range: {lower:.1f}-{upper:.1f}, avg: {avg_mpg:.1f} mpg)")

# Calculate expected team performance
total_expected_plus_minus = sum(l['duration'] * l['rating'] for l in lineups)
actual_team_plus_minus = sum(player_stats.values())

print(f"\n" + "-" * 70)
print(f"Total Expected Team +/- (Per Game):   {total_expected_plus_minus:+.2f}")
print(f"Actual Team +/- (Per Game):           {actual_team_plus_minus / 32:+.2f}")
print(f"Improvement per game:                 {total_expected_plus_minus - (actual_team_plus_minus / 32):+.2f}")

Player Rankings by +/- per minute:
 1. Patrick Wessler      +0.0604 per min  (10.7 mpg over 31 games)
     Minute range: 5.7-15.7
     Can play positions: 5
 2. Connor Serven        +0.0488 per min  (2.6 mpg over 16 games)
     Minute range: 0.0-7.6
     Can play positions: 4, 3, 5
 3. Tobi Lawal           +0.0000 per min  (26.4 mpg over 30 games)
     Minute range: 21.4-31.4
     Can play positions: 4, 5
 4. Rodney Brown Jr.     +0.0000 per min  (15.2 mpg over 18 games)
     Minute range: 10.2-20.2
     Can play positions: 3, 2
 5. Ben Burnham          -0.0199 per min  (22.0 mpg over 32 games)
     Minute range: 17.0-27.0
     Can play positions: 4
 6. Brandon Rechsteiner  -0.0212 per min  (22.2 mpg over 32 games)
     Minute range: 17.2-27.2
     Can play positions: 1, 2
 7. Jaydon Young         -0.0350 per min  (22.3 mpg over 32 games)
     Minute range: 17.3-27.3
     Can play positions: 2, 3
 8. Tyler Johnson        -0.0404 per min  (25.5 mpg over 32 games)
     Minute range: 20.5